In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import re
import time
from datasets import load_dataset
from collections import Counter
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA L4


In [3]:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
model.to("cuda")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [4]:
print(next(model.parameters()).device)

cuda:0


In [5]:
train_df = pd.read_csv("train.csv")
print(train_df.shape)
train_df.head()

(5616, 5)


,labels,jd_cleaned,resume_cleaned,input_ids,attention_mask
0,1,lrs consulting services has been delivering th...,<<summarysenior lead release engineer speciali...,[ 101 1048 2869 10552 2578 2038 2042 127...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
1,2,"driven, empowering, transformative. academic p...",<<summaryleadership-oriented certified technol...,[ 101 5533 1010 7861 23948 1010 10938 80...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
2,1,position title: senior accountant organization...,<<profilededicated epidemiologist/data manager...,[ 101 2597 2516 1024 3026 17907 3029 10...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
3,1,"at farmer brothers, sales tax accountant will ...",<<summaryseasoned data architect adept at unde...,[ 101 2012 7500 3428 1010 4341 4171 179...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
4,0,job description i am actively seeking an exper...,<<summaryenergetic and personableadministrativ...,[ 101 3105 6412 1045 2572 8851 6224 20...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...


In [6]:
test_df = pd.read_csv("test.csv")
print(test_df.shape)
test_df.head()

(1759, 5)


,labels,jd_cleaned,resume_cleaned,input_ids,attention_mask
0,1,key responsibilities:create intricate wiring n...,<<summary7+ years of experience as a bi develo...,[ 101 3145 10198 1024 3443 17796 27930 61...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
1,1,personal development and becoming the best you...,<<professional backgroundanalyst versed in dat...,[ 101 3167 2458 1998 3352 1996 2190 20...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
2,1,"location: tampa, fl exp: 7-10 yrs spoc: tushar...",<<executive profilededicated professional with...,[ 101 3295 1024 9925 1010 13109 4654 23...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
3,1,"primary location: melbourne, florida v-soft co...","<<summarytyee highlightsmicrosoft excel, word,...",[ 101 3078 3295 1024 4940 1010 3516 10...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
4,1,at oregon specialty group the accounting & pay...,<<summaryeit certified engineer and astqb cert...,[ 101 2012 5392 12233 2177 1996 9529 10...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...


In [7]:
val_df = pd.read_csv("validation.csv")
print(val_df.shape)
val_df.head()

(625, 5)


,labels,jd_cleaned,resume_cleaned,input_ids,attention_mask
0,0,job description job title: salesforce communic...,<<summaryi am a computer engineer with over 12...,[ 101 3105 6412 3105 2516 1024 4341 148...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
1,1,"at farmer brothers, sales tax accountant will ...",<<profilehighly motivated sales associate with...,[ 101 2012 7500 3428 1010 4341 4171 179...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
2,0,"hello,greetings from devcare solutionsi got an...",<<professional summaryhighly motivated sales a...,[ 101 7592 1010 14806 2015 2013 16475 163...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
3,1,if you can handle the accounting responsibilit...,<<summarymy name is jessica claire and i am in...,[ 101 2065 2017 2064 5047 1996 9529 101...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
4,0,job purpose: perform designated tasks in the a...,<<summarycapable accountant successful at mana...,[ 101 3105 3800 1024 4685 4351 8518 19...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...


In [8]:
import re

def clean_and_convert(x):
    if isinstance(x, list):
        return x  # already clean
    
    # 1. Remove brackets if present
    x = x.replace("[", "").replace("]", "")
    
    # 2. Strip spaces
    x = x.strip()
    
    # 3. Split by ANY whitespaces (handles double/triple spaces)
    parts = re.split(r"\s+", x)
    
    # 4. Convert each token to integer
    return [int(p) for p in parts if p.isdigit()]


In [9]:
for df in [train_df, val_df, test_df]:
    df["input_ids"] = df["input_ids"].apply(clean_and_convert)
    df["attention_mask"] = df["attention_mask"].apply(clean_and_convert)


In [10]:
train_df.head()

,labels,jd_cleaned,resume_cleaned,input_ids,attention_mask
0,1,lrs consulting services has been delivering th...,<<summarysenior lead release engineer speciali...,"[101, 1048, 2869, 10552, 2578, 2038, 2042, 127...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,2,"driven, empowering, transformative. academic p...",<<summaryleadership-oriented certified technol...,"[101, 5533, 1010, 7861, 23948, 1010, 10938, 80...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1,position title: senior accountant organization...,<<profilededicated epidemiologist/data manager...,"[101, 2597, 2516, 1024, 3026, 17907, 3029, 102...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1,"at farmer brothers, sales tax accountant will ...",<<summaryseasoned data architect adept at unde...,"[101, 2012, 7500, 3428, 1010, 4341, 4171, 1790...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,0,job description i am actively seeking an exper...,<<summaryenergetic and personableadministrativ...,"[101, 3105, 6412, 1045, 2572, 8851, 6224, 2019...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [11]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)
val_ds   = Dataset.from_pandas(val_df)
test_ds  = Dataset.from_pandas(test_df)


In [12]:
cols = ["input_ids", "attention_mask", "labels"]

train_ds = train_ds.with_format("torch", columns=cols)
val_ds   = val_ds.with_format("torch", columns=cols)
test_ds  = test_ds.with_format("torch", columns=cols)


In [13]:
train_ds

Dataset({
    features: ['labels', 'jd_cleaned', 'resume_cleaned', 'input_ids', 'attention_mask'],
    num_rows: 5616
})

In [14]:
val_ds

Dataset({
    features: ['labels', 'jd_cleaned', 'resume_cleaned', 'input_ids', 'attention_mask'],
    num_rows: 625
})

In [15]:
test_ds

Dataset({
    features: ['labels', 'jd_cleaned', 'resume_cleaned', 'input_ids', 'attention_mask'],
    num_rows: 1759
})

In [16]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


In [17]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [18]:
from transformers import BertForSequenceClassification

num_labels = len(train_df["labels"].unique())

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels
)
model.gradient_checkpointing_enable() 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df["labels"]),
    y=train_df["labels"]
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda")


In [21]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import torch.nn.functional as F
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = np.argmax(logits, axis=1)

#     return {
#         "accuracy": accuracy_score(labels, preds),
#         "f1": f1_score(labels, preds, average="weighted")
#     }

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = F.softmax(torch.tensor(logits), dim=1).numpy()
    preds = np.argmax(probs, axis=1)

    try:
        auc = roc_auc_score(labels, probs, multi_class="ovr")
    except:
        auc = float("nan")

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "auc": auc,
    }


In [23]:
from transformers import TrainingArguments

# args = TrainingArguments(
#     output_dir="bert-model",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     logging_strategy="epoch",
#     num_train_epochs=5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     fp16=True,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     greater_is_better=True,
#     report_to="none",
# )

from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="bert-base-advanced",

    eval_strategy="steps",
    save_strategy="steps",

    eval_steps=1500,          # ← NO EARLY EVAL = NO AUC DIP
    save_steps=1500,          # ← save only when we evaluate
    save_total_limit=1,       # ← only best checkpoint kept
    load_best_model_at_end=True,

    logging_strategy="steps",
    logging_steps=300,

    num_train_epochs=8,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,

    learning_rate=2e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",

    weight_decay=0.01,
    label_smoothing_factor=0.05,

    fp16=True,
    report_to="none",
)


In [ ]:
# from transformers import EarlyStoppingCallback

# early_stop = EarlyStoppingCallback(
#     early_stopping_patience=2,
#     early_stopping_threshold=0.0005
# )

In [24]:
import torch.nn as nn
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [25]:
from transformers import Trainer

trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/scratch/local/19646357/ipykernel_1886863/835306993.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


In [26]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Auc
1500,1.024100,1.096135,0.352000,0.280870,0.665295
3000,0.844400,0.819765,0.584000,0.573107,0.774839
4500,0.711100,0.796444,0.627200,0.621215,0.823317
6000,0.681900,0.803719,0.656000,0.652492,0.846013
7500,0.596400,0.913062,0.680000,0.678213,0.849931
9000,0.526000,1.005476,0.705600,0.702821,0.863593
10500,0.507400,1.055119,0.702400,0.701124,0.862247


TrainOutput(global_step=11232, training_loss=0.718159294535971, metrics={'train_runtime': 896.3843, 'train_samples_per_second': 50.121, 'train_steps_per_second': 12.53, 'total_flos': 1.1821159631683584e+16, 'train_loss': 0.718159294535971, 'epoch': 8.0})

In [29]:
trainer.evaluate(val_ds)

{'eval_loss': 0.7964439392089844,
 'eval_accuracy': 0.6272,
 'eval_f1': 0.6212149159954387,
 'eval_auc': 0.8233165772474673,
 'eval_runtime': 2.5216,
 'eval_samples_per_second': 247.86,
 'eval_steps_per_second': 62.262,
 'epoch': 8.0}

In [30]:
trainer.evaluate(test_ds)

{'eval_loss': 1.1910086870193481,
 'eval_accuracy': 0.4974417282546902,
 'eval_f1': 0.4463645660401656,
 'eval_auc': 0.6419424847599166,
 'eval_runtime': 6.8957,
 'eval_samples_per_second': 255.088,
 'eval_steps_per_second': 63.808,
 'epoch': 8.0}